In [9]:
import pandas as pd
import re

# --- 1. Konfigurasi ---
# Nama file CSV Anda
file_gaji = 'gaji_dan_tkdd.xlsx - GAJI.csv'
file_tkdd = 'gaji_dan_tkdd.xlsx - TKDD.csv'
# Gunakan 'r' di depan string agar backslash (\) dibaca dengan benar
file_gaji = r'C:\Users\penof\Documents\artefak-skom\data-science\praktikum7\code\gaji_dan_tkdd.xlsx - GAJI.csv'
file_tkdd = r'C:\Users\penof\Documents\artefak-skom\data-science\praktikum7\code\gaji_dan_tkdd.xlsx - TKDD.csv'
# Jumlah baris di atas header yang perlu dilewati
rows_to_skip = 8 

# Nama kolom yang bersih (setelah kita rapikan)
col_nama_clean = 'Nama / Jabatan'
col_pph_clean = 'PPh'
col_no_gaji = 'No.'   # Kolom nomor di file Gaji untuk filter
col_no_tkdd = 'Nomor' # Kolom nomor di file TKDD untuk filter

# --- 2. Fungsi "Pembersih Nama" ---
# Fungsi ini untuk mengatasi 'MOCHDAR' vs 'M O C H D A R'
def normalize_name(name):
    """Membersihkan nama agar konsisten."""
    if not isinstance(name, str):
        return ""
    # 1. Ubah ke huruf besar
    name = name.upper()
    # 2. Hapus tanda baca seperti titik dan koma
    name = re.sub(r'[.,]', '', name)
    # 3. Hapus SEMUA spasi (termasuk di tengah nama)
    name = re.sub(r'\s+', '', name)
    return name

# --- 3. Membaca dan Membersihkan Data ---
try:
    print(f"Membaca file {file_gaji}...")
    df_gaji = pd.read_csv(file_gaji, skiprows=rows_to_skip)
    
    print(f"Membaca file {file_tkdd}...")
    df_tkdd = pd.read_csv(file_tkdd, skiprows=rows_to_skip)

    # --- PERBAIKAN 1: Membersihkan nama kolom ---
    # Ini memperbaiki masalah 'PPh ' (dengan spasi)
    df_gaji.columns = df_gaji.columns.str.strip()
    df_tkdd.columns = df_tkdd.columns.str.strip()
    print("Nama kolom telah dibersihkan (menghapus spasi ekstra).")
    
    # --- PERBAIKAN 2: Membuang baris 'sampah' (yang tidak punya No.) ---
    # Ini membuang baris jabatan dan baris kosong
    # .dropna() akan menghapus baris yang nilai di kolom 'No.' atau 'Nomor' nya kosong (NaN)
    df_gaji = df_gaji.dropna(subset=[col_no_gaji])
    df_tkdd = df_tkdd.dropna(subset=[col_no_tkdd])
    print("Baris-baris 'sampah' (seperti baris jabatan) telah dibuang.")

    print("Melanjutkan proses...")

    # --- 4. Proses Data Gaji (Data Utama) ---
    print(f"Jumlah baris bersih di GAJI: {len(df_gaji)}")
    # Buat 'Kunci_Nama' yang bersih untuk dicocokkan
    df_gaji['Kunci_Nama'] = df_gaji[col_nama_clean].apply(normalize_name)
    # Ubah PPh menjadi angka (jika ada yang error/kosong, jadikan 0)
    df_gaji[col_pph_clean] = pd.to_numeric(df_gaji[col_pph_clean], errors='coerce').fillna(0)
    # Ambil kolom yang perlu saja dan ganti nama kolom PPh
    df_gaji_clean = df_gaji[[col_nama_clean, 'Kunci_Nama', col_pph_clean]].rename(columns={col_pph_clean: 'PPh_Gaji'})
    

    # --- 5. Proses Data TKDD (Data Tambahan) ---
    print(f"Jumlah baris bersih di TKDD: {len(df_tkdd)}")
    # Buat 'Kunci_Nama' yang bersih untuk dicocokkan
    df_tkdd['Kunci_Nama'] = df_tkdd[col_nama_clean].apply(normalize_name)
    # Ubah PPh menjadi angka
    df_tkdd[col_pph_clean] = pd.to_numeric(df_tkdd[col_pph_clean], errors='coerce').fillna(0)
    # Ambil kolom yang perlu saja
    df_tkdd_clean = df_tkdd[['Kunci_Nama', col_pph_clean]].rename(columns={col_pph_clean: 'PPh_TKDD'})

    # Jika ada nama duplikat di TKDD, jumlahkan PPh-nya
    df_tkdd_agg = df_tkdd_clean.groupby('Kunci_Nama')['PPh_TKDD'].sum().reset_index()

    # --- 6. Gabungkan Data (Logika Utama Anda) ---
    print("Menggabungkan kedua data...")
    # 'how='left'' = Ambil semua 37 orang dari Gaji, lalu pasangkan data TKDD
    df_final = pd.merge(df_gaji_clean, df_tkdd_agg, on='Kunci_Nama', how='left')

    # --- 7. Hitung Total PPh (Logika "Jika tidak ada = 0") ---
    # .fillna(0) = Untuk 8 orang yang tidak ada di TKDD, ubah PPh_TKDD-nya dari kosong (NaN) menjadi 0
    df_final['PPh_TKDD'] = df_final['PPh_TKDD'].fillna(0)
    
    # Jumlahkan PPh Gaji + PPh TKDD (yang sudah diisi 0 jika kosong)
    df_final['Total_PPh'] = df_final['PPh_Gaji'] + df_final['PPh_TKDD']
    
    # Buang kolom 'Kunci_Nama' karena sudah tidak dipakai
    df_final = df_final.drop(columns=['Kunci_Nama'])

    # --- 8. Simpan Hasil ---
    output_file = 'rekapitulasi_total_pph.xlsx'
    df_final.to_excel(output_file, index=False)

    print("\n" + "="*50)
    print("🎉 PROSES FINAL SELESAI 🎉")
    print(f"Total {len(df_final)} baris data telah diproses.")
    print(f"File hasil telah disimpan sebagai: {output_file}")
    
    print("\nContoh 5 baris pertama dari hasil akhir yang bersih:")
    print(df_final.head())

except Exception as e:
    print(f"Terjadi error yang tidak terduga: {e}")

Membaca file C:\Users\penof\Documents\artefak-skom\data-science\praktikum7\code\gaji_dan_tkdd.xlsx - GAJI.csv...
Terjadi error yang tidak terduga: [Errno 2] No such file or directory: 'C:\\Users\\penof\\Documents\\artefak-skom\\data-science\\praktikum7\\code\\gaji_dan_tkdd.xlsx - GAJI.csv'
